In [468]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [469]:
base = pd.read_csv('iec6100429.csv')

In [470]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [471]:
x.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 182,Amostra 183,Amostra 184,Amostra 185,Amostra 186,Amostra 187,Amostra 188,Amostra 189,Amostra 190,Amostra 191
0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
1,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
2,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
4,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3


In [472]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(80,)

In [473]:
y = y.values.reshape(-1,1)
y.shape

(80, 1)

In [474]:
y = onehot.fit_transform(y).toarray()

In [475]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.4)

In [476]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((80, 191), (32, 191), (80, 6), (32, 6))

In [477]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [478]:
neuronios_oculta = 160
neuronios_oculta

160

In [479]:
neuronios_saida = y.shape[1]
neuronios_saida

6

In [480]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable_84:0' shape=(191, 160) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_85:0' shape=(160, 6) dtype=float32_ref>}

In [481]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_86:0' shape=(160,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_87:0' shape=(6,) dtype=float32_ref>}

In [482]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [483]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [484]:
modelo = mlp(xph, w, b)

In [485]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(erro)

In [486]:
batch_size = 8
batch_total = int(len(x_treinamento) / batch_size)
batch_total, (len(x_treinamento))

(6, 48)

In [487]:
x_batches = np.array_split(x_treinamento, batch_total)

In [488]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(15000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 43205.007161458336
Época: 501 erro: 8.501385927200317
Época: 1001 erro: 3.0735250115394592
Época: 1501 erro: 7.849672883360957
Época: 2001 erro: 7.898223876953125
Época: 2501 erro: 19.87908935546875
Época: 3001 erro: 4.061016241660279
Época: 3501 erro: 0.0
Época: 4001 erro: 0.0
Época: 4501 erro: 0.0
Época: 5001 erro: 0.0
Época: 5501 erro: 0.0
Época: 6001 erro: 0.0
Época: 6501 erro: 0.0
Época: 7001 erro: 0.0
Época: 7501 erro: 0.0
Época: 8001 erro: 0.0
Época: 8501 erro: 0.0
Época: 9001 erro: 0.0
Época: 9501 erro: 0.0
Época: 10001 erro: 0.0
Época: 10501 erro: 0.0
Época: 11001 erro: 0.0
Época: 11501 erro: 0.0
Época: 12001 erro: 0.0
Época: 12501 erro: 0.0
Época: 13001 erro: 0.0
Época: 13501 erro: 0.0
Época: 14001 erro: 0.0
Época: 14501 erro: 0.0
1.0


In [489]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))
    
y_teste2 = np.argmax(y_teste, 1)
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.9375